In [6]:
import pandas as pd
import numpy as np
import time
import os
import urllib.request
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
import pickle
import re
from sklearn.preprocessing import LabelEncoder
import joblib
from collections import Counter
from collections import defaultdict
import ast
import mysql.connector

from selenium.common.exceptions import StaleElementReferenceException
from selenium.common.exceptions import NoSuchElementException

chrome_options = Options()
chrome_options.add_argument('--headless')  # 웹 브라우저를 표시하지 않고 백그라운드에서 실행
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

In [36]:
import mysql.connector

conn = mysql.connector.connect(
    user='root',
    password='llsy13579!',
    host='localhost',
    port=3306,
    database='recommend_project'
)

DB_table_name = 'recommend_project'

cursor = conn.cursor()
use_query = f"USE {DB_table_name}"

cursor.execute(use_query)

set_query = "SET innodb_lock_wait_timeout = 28800;"

cursor.execute(set_query)

In [21]:
def prepare_problem_list():
    problem_info_query = f"SELECT problem_number, problem_algorithm From problem_information_table"

    cursor.execute(problem_info_query)

    problem_info = cursor.fetchall()

    problem_number_list = [i[0] for i in problem_info]
    problem_algorithm_list = [i[1] for i in problem_info]
    cleaned_data = []

    for item in problem_algorithm_list:

        # 문자열을 파싱하여 리스트로 변환
        if(item=='Not Exist Algorithm'):
            continue

        item_list = eval(item)

        # 중복 제거를 위해 set으로 변환
        unique_set = set(item_list)

        cleaned_data.extend(unique_set)

    C = Counter(cleaned_data)
    C_dict = dict(sorted(C.items(), key = lambda item: -item[1]))
    problem_algorithm_list = list(C_dict.keys())
    
    problem_other_result_form_query = f'SELECT other_result_form_1, other_result_form_2 FROM problem_information_table'

    cursor.execute(problem_other_result_form_query)
    problem_other_result_form = [(i[0],i[1]) for i in cursor.fetchall()]

    problem_other_result_form_1 = [int(i[0]) for i in problem_other_result_form]
    problem_other_result_form_2 = [int(i[1]) for i in problem_other_result_form]

    problem_other_result_form_list_1 = []
    problem_other_result_form_list_2 = []

    for i, v in enumerate(problem_number_list):
        
        if(problem_other_result_form_1[i] != 0):
            problem_other_result_form_list_1.append((v, problem_other_result_form_1[i]))
        if(problem_other_result_form_2[i] != 0):
            problem_other_result_form_list_2.append((v, problem_other_result_form_2[i]))

    problem_other_result_form_dict_1 = dict(problem_other_result_form_list_1)
    problem_other_result_form_dict_2 = dict(problem_other_result_form_list_2)

    return (problem_number_list, problem_algorithm_list, problem_other_result_form_dict_1, problem_other_result_form_dict_2)

In [18]:
prepare_problem_list()

([1000,
  1001,
  1002,
  1003,
  1004,
  1005,
  1006,
  1007,
  1008,
  1009,
  1010,
  1011,
  1012,
  1013,
  1014,
  1015,
  1016,
  1017,
  1018,
  1019,
  1020,
  1021,
  1022,
  1023,
  1024,
  1025,
  1026,
  1027,
  1028,
  1029,
  1030,
  1031,
  1032,
  1033,
  1034,
  1035,
  1036,
  1037,
  1038,
  1039,
  1040,
  1041,
  1042,
  1043,
  1044,
  1045,
  1046,
  1047,
  1048,
  1049,
  1050,
  1051,
  1052,
  1053,
  1054,
  1055,
  1056,
  1057,
  1058,
  1059,
  1060,
  1061,
  1062,
  1063,
  1064,
  1065,
  1066,
  1067,
  1068,
  1069,
  1070,
  1071,
  1072,
  1073,
  1074,
  1075,
  1076,
  1077,
  1078,
  1079,
  1080,
  1081,
  1082,
  1083,
  1084,
  1085,
  1086,
  1087,
  1088,
  1089,
  1090,
  1091,
  1092,
  1093,
  1094,
  1095,
  1096,
  1097,
  1098,
  1099,
  1100,
  1101,
  1102,
  1103,
  1104,
  1105,
  1106,
  1107,
  1108,
  1109,
  1110,
  1111,
  1112,
  1113,
  1114,
  1115,
  1116,
  1117,
  1118,
  1119,
  1120,
  1121,
  1122,
  1123,
  1124,


In [22]:
def crawling_user_information(user_id, problem_number_list):

    user_information_url = 'https://www.acmicpc.net/user/' + str(user_id)

    driver = webdriver.Chrome('../Crawling/chromedriver.exe', options=chrome_options)
    driver.get(user_information_url)


    user_solve_problem_list = []
    tier_value = -1
    rank_value = -1

    #유저 정보가 존재한다면, 유저 정보 긁어오기.
    try:
        user_solve_problem_object = driver.find_element_by_css_selector('.problem-list')
        user_solve_problem = user_solve_problem_object.text.split(' ')
        user_solve_problem_list = [int(problem) for problem in user_solve_problem if int(problem) in problem_number_list]
    
    #존재하지 않는다면, 오류 메세지 출력
    except NoSuchElementException:
        print('해당 유저가 존재하지 않습니다.')
        return -1

    try:
        # id가 "statics"인 테이블 찾기
        table_element = driver.find_element_by_css_selector("#statics")
        
        # 테이블 내에서 '등수' 텍스트를 포함한 <tr> 찾기
        for t in table_element.text.split('\n'):
            if('등수' in t):
                rank_value = int(t.split(' ')[1])
                break

    except NoSuchElementException:
        print('해당 유저의 랭크가 존재하지 않습니다.')

    try:
        solvedac_tier_element = driver.find_element_by_css_selector(".solvedac-tier")
        img_src = solvedac_tier_element.get_attribute("src")

        pattern = r'/tier/(\d+)\.svg'

        match = re.search(pattern, img_src)

        tier_value = int(match.group(1))
    except NoSuchElementException:
        print('해당 유저의 티어가 존재하지 않습니다.')


    driver.quit()

    return (tier_value, rank_value, user_solve_problem_list, len(user_solve_problem_list))

In [33]:
crawling_user_information('xiaowuc1')

15009


(31,
 1,
 [1000,
  1001,
  1002,
  1003,
  1004,
  1005,
  1006,
  1007,
  1008,
  1009,
  1010,
  1011,
  1012,
  1013,
  1014,
  1015,
  1016,
  1017,
  1018,
  1019,
  1020,
  1021,
  1022,
  1023,
  1024,
  1025,
  1026,
  1027,
  1028,
  1029,
  1030,
  1031,
  1032,
  1033,
  1034,
  1035,
  1036,
  1037,
  1038,
  1039,
  1040,
  1041,
  1043,
  1044,
  1045,
  1047,
  1049,
  1050,
  1051,
  1052,
  1053,
  1054,
  1055,
  1056,
  1057,
  1058,
  1059,
  1060,
  1061,
  1062,
  1063,
  1064,
  1065,
  1067,
  1068,
  1069,
  1071,
  1072,
  1074,
  1075,
  1076,
  1079,
  1080,
  1081,
  1082,
  1083,
  1085,
  1086,
  1088,
  1089,
  1091,
  1092,
  1094,
  1095,
  1099,
  1100,
  1101,
  1102,
  1103,
  1105,
  1106,
  1107,
  1108,
  1110,
  1111,
  1112,
  1113,
  1114,
  1115,
  1117,
  1119,
  1120,
  1121,
  1122,
  1124,
  1126,
  1131,
  1132,
  1135,
  1136,
  1138,
  1139,
  1141,
  1143,
  1144,
  1145,
  1146,
  1148,
  1149,
  1150,
  1152,
  1153,
  1156,
  1157,

In [23]:
def crawling_user_problem_interaction(user_id):

    user_problem_interaction_url = 'https://www.acmicpc.net/status?user_id=' + str(user_id)
    driver = webdriver.Chrome('../Crawling/chromedriver.exe', options=chrome_options)
    driver.get(user_problem_interaction_url)

    submit_number_list = []
    user_id_list = []
    problem_number_list = []
    problem_result_list = []
    code_memory_list = []
    code_time_list = []
    code_language_list = []


    flag = True

    while(True):

        user_data = driver.find_elements_by_css_selector('[data-can-view]')
        
        try:
            user_next_page = driver.find_element_by_css_selector('#next_page')

        except NoSuchElementException:
            flag = False

        for user_info in user_data:
        
            user_text = user_info.find_elements_by_css_selector("td")
            user_text_list = []

            for ut in user_text:
                user_text_list.append(ut.text)
            
            submit_number_list.append(user_text_list[0])
            user_id_list.append(user_text_list[1])
            problem_number_list.append(user_text_list[2])
            problem_result_list.append(user_text_list[3])
            code_memory_list.append(np.nan if user_text_list[4]=='' else user_text_list[4])
            code_time_list.append(np.nan if user_text_list[5]=='' else user_text_list[5])
            code_language_list.append(user_text_list[6])

        if(flag):
            user_next_page.click()
        else:
            break            
    
    user_problem_interaction_information = pd.DataFrame({
        'submit_number_list': submit_number_list,
        'user_id_list': user_id_list,
        'problem_number_list': problem_number_list,
        'problem_result_list': problem_result_list,
        'code_memory_list': code_memory_list,
        'code_time_list': code_time_list,
        'code_language_list': code_language_list,
    })

    driver.quit()
    
    return user_problem_interaction_information

In [24]:
def user_problem_interaction_preprocessing(user_problem_interaction_df, problem_number_list):
    # 유저 아이디 목록이 존재하지 않는 것 존재 -> 데이터 오류, 삭제 요망
    user_problem_interaction_df = user_problem_interaction_df[user_problem_interaction_df['user_id_list'] == user_problem_interaction_df['user_id_list']]
    user_problem_interaction_df = user_problem_interaction_df[user_problem_interaction_df['code_language_list'] == user_problem_interaction_df['code_language_list']]

    # Problem number 형 변환
    # memory, time은 틀렸을 시 nan값 -> 결측치는 아니므로 -1 로 대체

    user_problem_interaction_df['problem_number_list'] = user_problem_interaction_df['problem_number_list'].astype(int)
    user_problem_interaction_df['code_memory_list'].fillna(-1, inplace=True)
    user_problem_interaction_df['code_time_list'].fillna(-1, inplace=True)
    user_problem_interaction_df['code_memory_list'] = user_problem_interaction_df['code_memory_list'].astype(int)
    user_problem_interaction_df['code_time_list'] = user_problem_interaction_df['code_time_list'].astype(int)

    user_problem_interaction_df = user_problem_interaction_df[user_problem_interaction_df['problem_result_list']!='채점 불가']
    user_problem_interaction_df = user_problem_interaction_df[user_problem_interaction_df['problem_result_list']!='채점 중']

    #DB에 없는 문제 거르는 작업
    user_problem_interaction_df = user_problem_interaction_df[user_problem_interaction_df['problem_number_list'].isin(problem_number_list)]

    return user_problem_interaction_df

In [39]:
def user_problem_interaction_to_DB_form(user_id, user_problem_interaction_df, problem_algorithm_list, problem_other_result_form_dict_1, problem_other_result_form_dict_2):

    total_result_dict = dict()
    user_weak_algorithm = []
    user_failed_problem = []
    
    user_algorithm_dict=dict(zip(problem_algorithm_list, [0]*len(problem_algorithm_list)))
    user_problem_interaction_df_groupby_problem_number = user_problem_interaction_df.groupby('problem_number_list')

    other_form_1_key = problem_other_result_form_dict_1.keys()
    other_form_2_key = problem_other_result_form_dict_2.keys()

    for problem_number, row in user_problem_interaction_df_groupby_problem_number:
        
        problem_algorithm_query = 'SELECT problem_algorithm FROM problem_information_table WHERE problem_number=%s'
        cursor.execute(problem_algorithm_query, (problem_number,))
        sub_problem_algorithm = cursor.fetchall()

        result_dict = {
            'user_problem_key' : 0,
            'user_id' : 0,
            'problem_number' : 0,
            'correct_attempts_count' : 0,
            'incorrect_attempts_count' : 0,
            'time_limit_exceeded_count' : 0,
            'memory_limit_exceeded_count' : 0,
            'output_limit_exceeded_count' : 0,
            'runtime_error_count' : 0,
            'compile_error_count' : 0,
            'output_format_issue_count' : 0,
            'most_used_language' : 0,
            'best_memory_usage' : 0,
            'fastest_execution_time' : 0,
            'memory_limit_optimal_solution_submission_number' : 0,
            'time_limit_optimal_solution_submission_number' : 0
        }

        correct_attempts_list = []

        result_dict['user_problem_key'] = str(user_id) + '_' + str(problem_number)
        result_dict['user_id'] = user_id
        result_dict['problem_number'] = problem_number
        
        for index, r in row.iterrows():
            
            flag = False
            result = r['problem_result_list']

            if(problem_number in other_form_1_key):
                if('점' in result):
                    result_score = re.search(r'\d+(\.\d+)?', result).group()
                    if(float(result_score) if '.' in result_score else int(result_score) == problem_other_result_form_dict_1[problem_number]):
                        result_dict['correct_attempts_count'] += 1
                        flag = True
                    else:
                        result_dict['incorrect_attempts_count'] += 1
                elif('틀렸습니다' in result):
                    result_dict['incorrect_attempts_count'] += 1
                elif('시간 초과' in result):
                    result_dict['time_limit_exceeded_count'] += 1
                elif('메모리 초과' in result):
                    result_dict['memory_limit_exceeded_count'] += 1
                elif('출력 초과' in result):
                    result_dict['output_limit_exceeded_count'] += 1
                elif('런타임 에러' in result):
                    result_dict['runtime_error_count'] += 1
                elif('컴파일 에러' in result):
                    result_dict['compile_error_count'] += 1
                elif('출력 형식이 잘못되었습니다' in result):
                    result_dict['output_format_issue_count'] += 1
                else:
                    pass
            elif(problem_number in other_form_2_key):
                if('맞았습니다!!' in result):
                    numbers = re.findall(r'\d+', result)
                    if(numbers[0] == numbers[1]):
                        result_dict['correct_attempts_count'] += 1
                        flag = True
                    elif(numbers[0] != numbers[1]):
                        result_dict['incorrect_attempts_count'] += 1
                elif('틀렸습니다' in result):
                    result_dict['incorrect_attempts_count'] += 1
                elif('시간 초과' in result):
                    result_dict['time_limit_exceeded_count'] += 1
                elif('메모리 초과' in result):
                    result_dict['memory_limit_exceeded_count'] += 1
                elif('출력 초과' in result):
                    result_dict['output_limit_exceeded_count'] += 1
                elif('런타임 에러' in result):
                    result_dict['runtime_error_count'] += 1
                elif('컴파일 에러' in result):
                    result_dict['compile_error_count'] += 1
                elif('출력 형식이 잘못되었습니다' in result):
                    result_dict['output_format_issue_count'] += 1
                else:
                    pass
            else:
                if('맞았습니다!!' in result):
                    result_dict['correct_attempts_count'] += 1
                    flag = True
                elif('틀렸습니다' in result):
                    result_dict['incorrect_attempts_count'] += 1
                elif('시간 초과' in result):
                    result_dict['time_limit_exceeded_count'] += 1
                elif('메모리 초과' in result):
                    result_dict['memory_limit_exceeded_count'] += 1
                elif('출력 초과' in result):
                    result_dict['output_limit_exceeded_count'] += 1
                elif('런타임 에러' in result):
                    result_dict['runtime_error_count'] += 1
                elif('컴파일 에러' in result):
                    result_dict['compile_error_count'] += 1
                elif('출력 형식이 잘못되었습니다' in result):
                    result_dict['output_format_issue_count'] += 1
                else:
                    pass

            if(flag):
                correct_attempts_list.append((r['submit_number_list'], r['code_memory_list'], r['code_time_list'], r['code_language_list']))
            else:
                if(sub_problem_algorithm != 'Not Exist Algorithm'):
                    for s in eval(sub_problem_algorithm[0][0]):
                        user_algorithm_dict[s] += 1

        

        if(result_dict['correct_attempts_count']==0):
            user_failed_problem.append(problem_number)
        
        else:
            frequency_counter = Counter(tuple_[3] for tuple_ in correct_attempts_list)
            most_common_language = frequency_counter.most_common(1)[0][0]
            filtered_tuples = [tuple_ for tuple_ in correct_attempts_list if tuple_[3] == most_common_language]

            best_memory = min(filtered_tuples, key=lambda tuple_: tuple_[1])
            best_time = min(filtered_tuples, key=lambda tuple_: tuple_[2])

            result_dict['most_used_language'] = most_common_language
            result_dict['best_memory_usage'] = best_memory[1]
            result_dict['fastest_execution_time'] = best_time[2]
            result_dict['memory_limit_optimal_solution_submission_number'] = best_memory[0]
            result_dict['time_limit_optimal_solution_submission_number'] = best_time[0]
        
        total_result_dict[problem_number] = result_dict

    user_algorithm_dict = dict(sorted(user_algorithm_dict.items(), key = lambda item: item[1],reverse=True))
    user_weak_algorithm = list(user_algorithm_dict.keys())[:10]
    
    return (total_result_dict, user_failed_problem, user_weak_algorithm)
        

In [26]:
def new_user_process(user_id, user_tier, user_rank, user_solved_problem, user_failed_problem, user_solved_problem_number, user_problem_interaction_info):

    #유저 테이블 데이터 전처리 및 DB 저장 함수 실행. (Insert)
    #- 유저 id가 없으므로, Insert 실행

    user_id_insert_query = "INSERT INTO user_information_table (user_id, user_tier, user_rank, user_solved_problem_list, user_failed_problem_list, user_solved_problem_count) VALUES (%s, %s, %s, %s, %s, %s);"
    cursor.execute(user_id_insert_query, (user_id, user_tier, user_rank, str(user_solved_problem), str(user_failed_problem), user_solved_problem_number))
    conn.commit()

    #유저 - 문제 상호작용 테이블 데이터 전처리 및 DB 저장 함수 실행. (Insert)
    #- 유저 - 문제 Key에서 유저 자체가 없으므로, 모든 정보 Insert

    user_problem_interaction_insert_query = "INSERT INTO user_problem_interaction_table (user_problem_key ,user_id ,problem_number ,correct_attempts_count ,incorrect_attempts_count ,time_limit_exceeded_count ,memory_limit_exceeded_count ,output_limit_exceeded_count ,runtime_error_count ,compile_error_count ,output_format_issue_count ,most_used_language ,best_memory_usage ,fastest_execution_time ,memory_limit_optimal_solution_submission_number ,time_limit_optimal_solution_submission_number) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
    for i, v in user_problem_interaction_info.items():
        cursor.execute(user_problem_interaction_insert_query, (v['user_problem_key'], v['user_id'], v['problem_number'], v['correct_attempts_count'], v['incorrect_attempts_count'], v['time_limit_exceeded_count'], v['memory_limit_exceeded_count'], v['output_limit_exceeded_count'], v['runtime_error_count'], v['compile_error_count'], v['output_format_issue_count'], v['most_used_language'], v['best_memory_usage'], v['fastest_execution_time'], v['memory_limit_optimal_solution_submission_number'], v['time_limit_optimal_solution_submission_number']))
    
    conn.commit()
    


In [27]:
def regular_user_process(user_id, user_tier, user_rank, user_solved_problem, user_failed_problem, user_solved_problem_number, user_problem_interaction_info):

    #유저 테이블 데이터 전처리 및 DB 저장 함수 실행. (Update)
    #- 유저 id 찾아서 업데이트
    
    user_table_update_query = "UPDATE user_information_table SET user_tier = %s, user_rank = %s, user_solved_problem_list = %s, user_failed_problem_list = %s, user_solved_problem_count = %s WHERE user_id = %s;"
    cursor.execute(user_table_update_query, (user_tier, user_rank, str(user_solved_problem), str(user_failed_problem), user_solved_problem_number, user_id))
    conn.commit()

    #유저 - 문제 상호작용 테이블 데이터 전처리 및 DB 저장 함수 실행. (Update)
    #- 유저 - 문제 Key가 있다면, Update 없다면, Insert

    user_problem_interaction_insert_query = "INSERT INTO user_problem_interaction_table (user_problem_key ,user_id ,problem_number ,correct_attempts_count ,incorrect_attempts_count ,time_limit_exceeded_count ,memory_limit_exceeded_count ,output_limit_exceeded_count ,runtime_error_count ,compile_error_count ,output_format_issue_count ,most_used_language ,best_memory_usage ,fastest_execution_time ,memory_limit_optimal_solution_submission_number ,time_limit_optimal_solution_submission_number) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
    user_problem_interaction_update_query = "UPDATE user_problem_interaction_table SET correct_attempts_count = %s, incorrect_attempts_count = %s, time_limit_exceeded_count = %s, memory_limit_exceeded_count = %s, output_limit_exceeded_count = %s, runtime_error_count = %s, compile_error_count = %s, output_format_issue_count = %s, most_used_language = %s, best_memory_usage = %s, fastest_execution_time = %s, memory_limit_optimal_solution_submission_number = %s, time_limit_optimal_solution_submission_number = %s WHERE user_problem_key = %s;"              
    
    for i, v in user_problem_interaction_info.items():

        up_key = v['user_problem_key']

        check_user_problem_key_query = f'SELECT EXISTS (SELECT TRUE FROM user_problem_interaction_table WHERE user_problem_key = \'{up_key}\') AS value_exists;'
    
        cursor.execute(check_user_problem_key_query)
        
        check_user_problem_key_result = cursor.fetchone()

        if(check_user_problem_key_result[0] == 1):
            cursor.execute(user_problem_interaction_update_query, (v['correct_attempts_count'], v['incorrect_attempts_count'], v['time_limit_exceeded_count'], v['memory_limit_exceeded_count'], v['output_limit_exceeded_count'], v['runtime_error_count'], v['compile_error_count'], v['output_format_issue_count'], v['most_used_language'], v['best_memory_usage'], v['fastest_execution_time'], v['memory_limit_optimal_solution_submission_number'], v['time_limit_optimal_solution_submission_number'], v['user_problem_key']))
        else:
            cursor.execute(user_problem_interaction_insert_query, (v['user_problem_key'], v['user_id'], v['problem_number'], v['correct_attempts_count'], v['incorrect_attempts_count'], v['time_limit_exceeded_count'], v['memory_limit_exceeded_count'], v['output_limit_exceeded_count'], v['runtime_error_count'], v['compile_error_count'], v['output_format_issue_count'], v['most_used_language'], v['best_memory_usage'], v['fastest_execution_time'], v['memory_limit_optimal_solution_submission_number'], v['time_limit_optimal_solution_submission_number']))
    
    conn.commit()
    

In [45]:
a = crawling_user_problem_interaction('llsy159')

In [73]:
a = user_problem_interaction_preprocessing(a)

In [56]:
problem_number_list, problem_other_result_form_dict_1, problem_other_result_form_dict_2 = prepare_problem_list()

In [77]:
user_problem_interaction_to_DB_form('llsy159', a, problem_other_result_form_dict_1, problem_other_result_form_dict_2)

({1000: {'user_problem_key': 'llsy159_1000',
   'user_id': 'llsy159',
   'problem_number': 1000,
   'correct_attempts_count': 1,
   'incorrect_attempts_count': 0,
   'time_limit_exceeded_count': 0,
   'memory_limit_exceeded_count': 0,
   'output_limit_exceeded_count': 0,
   'runtime_error_count': 0,
   'compile_error_count': 0,
   'output_format_issue_count': 0,
   'most_used_language': 'Python 3',
   'best_memory_usage': 43004,
   'fastest_execution_time': 160,
   'memory_limit_optimal_solution_submission_number': '45160950',
   'time_limit_optimal_solution_submission_number': '45160950'},
  1003: {'user_problem_key': 'llsy159_1003',
   'user_id': 'llsy159',
   'problem_number': 1003,
   'correct_attempts_count': 2,
   'incorrect_attempts_count': 0,
   'time_limit_exceeded_count': 2,
   'memory_limit_exceeded_count': 0,
   'output_limit_exceeded_count': 0,
   'runtime_error_count': 0,
   'compile_error_count': 0,
   'output_format_issue_count': 0,
   'most_used_language': 'Java 15',
 

In [29]:
#렉걸려서 잠시 연결 안된 예외도 처리해주자.

def user_update(user_id):

    problem_number_list, problem_algorithm_list, problem_other_result_form_dict_1, problem_other_result_form_dict_2 = prepare_problem_list()
    
    #유저 테이블에 필요한 정보 크롤링하는 함수 실행.
    user_information = crawling_user_information(user_id,problem_number_list)
    
    #유저가 존재하지 않는다면,
    if(user_information == -1):
        print('해당 유저가 존재하지 않습니다.')
        #해당 유저의 데이터 삭제하는 코드 실행.

        #근데, 존재하지 않으면 데이터를 집어넣을 일이 없고, 이미 넣어진 상태에서 제거된다는 것은 유저가 탈퇴한 경우인데, 
        #AI의 인풋 데이터가 될 수도 있는 것을 굳이 삭제할 필요가 있을까?... 

        return -1

    #유저의 제출들을 크롤링하는 함수 실행.
    user_problem_interaction = crawling_user_problem_interaction(user_id)

    # user_problem_interaction 전처리 함수 실행

    user_problem_interaction = user_problem_interaction_preprocessing(user_problem_interaction, problem_number_list)
    user_problem_interaction = user_problem_interaction_to_DB_form(user_id, user_problem_interaction, problem_algorithm_list, problem_other_result_form_dict_1, problem_other_result_form_dict_2)
    
    print(user_information)
    print(user_problem_interaction)

    user_tier = user_information[0]
    user_rank = user_information[1]
    user_solved_problem = user_information[2]
    user_solved_problem_number = user_information[3]

    user_problem_interaction_info = user_problem_interaction[0]
    user_failed_problem = user_problem_interaction[1]
    user_weak_algorithm = user_problem_interaction[2]

    """

    # DB의 유저 테이블에 해당 유저가 존재하는지 확인.

    check_user_id_query = f'SELECT EXISTS (SELECT TRUE FROM user_information_table WHERE user_id = \'{user_id}\') AS value_exists;'
    
    cursor.execute(check_user_id_query)
    
    check_user_id_result = cursor.fetchone()
    

    # 있다면, 해당 유저의 정보를 갱신.
    if(check_user_id_result[0] == 1):
        regular_user_process(user_id, user_tier, user_rank, user_solved_problem, user_failed_problem, user_solved_problem_number, user_problem_interaction_info)
        print('기존 유저 갱신')
        
        
    # 없다면, 유저의 정보를 추가
    else:
        new_user_process(user_id, user_tier, user_rank, user_solved_problem, user_failed_problem, user_solved_problem_number, user_problem_interaction_info)
        print('신규 유저 추가')
    
    cursor.close()
    conn.close()
    """


In [40]:
user_update('0_o')

(17, 3979, [1000, 1001, 1002, 1003, 1005, 1008, 1009, 1010, 1011, 1012, 1018, 1026, 1037, 1057, 1059, 1065, 1074, 1075, 1085, 1094, 1100, 1106, 1107, 1110, 1149, 1152, 1153, 1157, 1158, 1181, 1182, 1193, 1197, 1202, 1209, 1238, 1259, 1260, 1271, 1309, 1316, 1330, 1406, 1427, 1436, 1463, 1475, 1476, 1495, 1509, 1517, 1520, 1541, 1546, 1550, 1620, 1629, 1644, 1647, 1654, 1655, 1676, 1689, 1697, 1699, 1712, 1715, 1720, 1725, 1726, 1735, 1753, 1757, 1761, 1764, 1780, 1786, 1788, 1806, 1850, 1874, 1890, 1904, 1912, 1915, 1920, 1924, 1927, 1929, 1931, 1932, 1934, 1937, 1964, 1965, 1966, 1976, 1977, 1978, 1987, 1991, 1992, 2010, 2011, 2042, 2095, 2096, 2098, 2108, 2110, 2133, 2156, 2162, 2163, 2164, 2166, 2167, 2178, 2193, 2225, 2231, 2240, 2252, 2292, 2293, 2294, 2302, 2309, 2338, 2357, 2417, 2420, 2436, 2438, 2439, 2440, 2441, 2442, 2443, 2444, 2445, 2446, 2447, 2448, 2455, 2464, 2467, 2470, 2473, 2475, 2480, 2482, 2490, 2493, 2512, 2522, 2523, 2525, 2540, 2541, 2545, 2557, 2558, 2562, 2565

In [19]:
check_user_id_query = f'SELECT * FROM user_problem_interaction_table;'
    
cursor.execute(check_user_id_query)
    
check_user_id_result = cursor.fetchall()

In [1]:
import mysql.connector

conn = mysql.connector.connect(
    user='root',
    password='llsy13579!',
    host='localhost',
    port=3306,
    database='recommend_project'
)

DB_table_name = 'recommend_project'

cursor = conn.cursor()
use_query = f"USE {DB_table_name}"

cursor.execute(use_query)

set_query = "SET innodb_lock_wait_timeout = 28800;"

cursor.execute(set_query)

In [20]:
len(check_user_id_result)

5046145


user_information_table 컬럼 정보

'user_id' = 유저의 아이디
'user_tier' = 유저의 티어
'user_rank' = 유저의 랭크(티어가 존재하지 않는 유저의 실력을 가늠하기 위함)
'user_solved_problem_list' = 유저가 해결한 모든 문제의 번호 리스트 (problem_information_table에 존재하는 문제들만 존재.)
'user_failed_problem_list' = 유저가 시도했지만 맞추지 못한 문제의 리스트 (problem_information_table에 존재하는 문제들만 존재.)
'user_solved_problem_count' = 유저가 해결한 모든 문제의 수

problem_information_table 컬럼 정보

'problem_number' = 문제 번호
'problem_name' = 문제 이름
'problem_tier' = 문제 티어
'problem_algorithm' = 문제 알고리즘
'problem_time_condition' = 문제 시간 조건
'problem_memory_condition' = 문제 메모리 조건
'problem_submission_count' = 문제의 총 제출 횟수
'problem_answer_submission_count' = 총 정답 제출 횟수
'problem_answered_people_count' = 맞힌 사람 수
'problem_correct_rate' = 정답률
'problem_description' = 문제 설명
'problem_input' = 문제의 input 형식
'problem_output' = 문제의 output 형식
'other_result_form_1' = 점수제 문제에 대한 만점의 점수  ex) 0이면 점수제 문제가 아님! 일반적인 문제인 것. 0이 아니라면 점수제 문제, 만약 값이 100이라면 만점은 100점인 것.
'other_result_form_2' = 맞았습니다 비율제 문제에 대한 비율값 ex) 0이면 맞았습니다 비율제 문제가 아님! 일반적인 문제인 것. 0이 아니라면 비율제 문제 값이 166이라면, 맞았습니다(30/166)이나 맞았습니다(166/166) 이런식으로 표현되는 문제인 것.


user_problem_interaction_table 컬럼 정보

'user_problem_key' = 유저의 id와 해당 유저가 푼 문제 정보에 대한 고유 식별자. ex) llsy159_2013 이라면, llsy159 유저가 2013번을 푼 정보에 대한 행임을 나타내는 고유 식별자(DB 값 조회 등 여러 방면에서 쓰임)
'user_id' = 유저의 id값
'problem_number' = 해당 유저가 푼 문제의 번호
'correct_attempts_count' = 맞힌 횟수
'incorrect_attempts_count' = 틀린 횟수
'time_limit_exceeded_count' = 시간 초과 횟수
'memory_limit_exceeded_count' = 메모리 초과 횟수
'output_limit_exceeded_count' = 출력 초과 횟수
'runtime_error_count' = 런타임 에러 횟수
'compile_error_count' = 컴파일 에러 횟수
'output_format_issue_count' = 출력 형식이 잘못된 제출 횟수 
'most_used_language' = 이 문제를 풀면서 맞은 기록 중에 가장 많이 사용한 언어 횟수(이 문제를 맞히는데 사용한 언어)
'best_memory_usage' = 맞힌 기록 중에 가장 효율적인 메모리 양
'fastest_execution_time' = 맞힌 기록중에 가장 효율적인 시간
'memory_limit_optimal_solution_submission_number' = 가장 효율적으로 메모리를 사용한 코드의 제출 번호
'time_limit_optimal_solution_submission_number' = 가장 시간이 적게 걸린 코드의 제출 번호

In [22]:
result = pd.DataFrame(check_user_id_result,columns=['user_id', 
    'user_tier',
    'user_rank',
    'user_solved_problem_list',
    'user_failed_problem_list',
    'user_solved_problem_count'])
result

,user_problem_key,user_id,problem_number,correct_attempts_count,incorrect_attempts_count,time_limit_exceeded_count,memory_limit_exceeded_count,output_limit_exceeded_count,runtime_error_count,compile_error_count,output_format_issue_count,most_used_language,best_memory_usage,fastest_execution_time,memory_limit_optimal_solution_submission_number,time_limit_optimal_solution_submission_number
0,0_o_1000,0_o,1000,3,0,0,0,0,0,0,0,C++17,1116,0,13462587,13462587
1,0_o_1001,0_o,1001,2,1,0,0,0,0,1,0,Boo,4352,180,13015410,13015410
2,0_o_1002,0_o,1002,1,5,0,0,0,0,0,0,C++17,1128,0,11959544,11959544
3,0_o_10026,0_o,10026,1,0,0,0,0,0,0,0,C++17,2152,0,13219485,13219485
4,0_o_1003,0_o,1003,1,0,0,0,0,0,0,0,C++17,1112,0,11564220,11564220
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5046140,zzzzll_9656,zzzzll,9656,1,2,0,0,0,1,0,0,C++17,2020,0,40732810,40732810
5046141,zzzzll_9657,zzzzll,9657,1,6,0,0,0,0,0,0,C++17,2020,0,44046623,44046623
5046142,zzzzll_9663,zzzzll,9663,1,0,1,0,0,0,0,0,C++17,2020,6044,31567002,31567002
5046143,zzzzll_9935,zzzzll,9935,2,3,0,0,0,0,0,0,C++17,12272,84,41696457,41696457


In [23]:
result.to_csv('user_problem_interaction_table.csv', index=False)

teddybear1024

일주일동안 저희 서비스를 사용한 사용자 이름을 쫙 모읍니다.

그 사람들에 대해서만 최신화를 진행

In [ ]:
1. 알고리즘 별 추천

닉네임이랑 알고리즘을 같이.

In [3]:
experiment_query = f"SELECT problem_algorithm FROM problem_information_table;"

cursor.execute(experiment_query)
 
problem_algorithm_result = cursor.fetchall()

In [4]:
problem_algorithm_result

[("['수학', '구현', '사칙연산']",),
 ("['수학', '구현', '사칙연산']",),
 ("['수학', '기하학', '많은 조건 분기']",),
 ("['다이나믹 프로그래밍']",),
 ("['수학', '기하학']",),
 ("['다이나믹 프로그래밍', '그래프 이론', '위상 정렬', '방향 비순환 그래프']",),
 ("['다이나믹 프로그래밍']",),
 ("['수학', '브루트포스 알고리즘']",),
 ("['수학', '구현', '사칙연산']",),
 ("['수학', '구현']",),
 ("['수학', '다이나믹 프로그래밍', '조합론']",),
 ("['수학']",),
 ("['그래프 이론', '그래프 탐색', '너비 우선 탐색', '깊이 우선 탐색']",),
 ("['문자열', '정규 표현식']",),
 ("['다이나믹 프로그래밍', '비트마스킹', '최대 유량', '비트필드를 이용한 다이나믹 프로그래밍']",),
 ("['정렬']",),
 ("['수학', '정수론', '소수 판정', '에라토스테네스의 체']",),
 ("['수학', '정수론', '소수 판정', '에라토스테네스의 체', '이분 매칭']",),
 ("['브루트포스 알고리즘']",),
 ("['수학']",),
 ("['다이나믹 프로그래밍']",),
 ("['자료 구조', '덱']",),
 ("['수학', '구현']",),
 ("['수학', '다이나믹 프로그래밍', '조합론']",),
 ("['수학']",),
 ("['브루트포스 알고리즘']",),
 ("['수학', '그리디 알고리즘', '정렬']",),
 ("['수학', '브루트포스 알고리즘', '기하학']",),
 ("['다이나믹 프로그래밍', '누적 합']",),
 ("['다이나믹 프로그래밍', '비트마스킹', '비트필드를 이용한 다이나믹 프로그래밍']",),
 ("['구현', '분할 정복', '재귀']",),
 ("['최대 유량']",),
 ("['구현', '문자열']",),
 ("['수학', '그래프 이론', '그래프

In [16]:

cleaned_data = []

for item in problem_algorithm_result:
    # 문자열을 파싱하여 리스트로 변환
    if(item[0]=='Not Exist Algorithm'):
        continue

    item_list = eval(item[0])

    # 중복 제거를 위해 set으로 변환
    unique_set = set(item_list)

    cleaned_data.extend(unique_set)

C = Counter(cleaned_data)
C_dict = dict(sorted(C.items(), key = lambda item: -item[1]))
C_dict

{'수학': 2515,
 '구현': 2033,
 '다이나믹 프로그래밍': 1687,
 '자료 구조': 1468,
 '그래프 이론': 1414,
 '그리디 알고리즘': 922,
 '문자열': 825,
 '브루트포스 알고리즘': 812,
 '그래프 탐색': 807,
 '정렬': 728,
 '정수론': 631,
 '트리': 581,
 '세그먼트 트리': 530,
 '애드 혹': 505,
 '기하학': 466,
 '이분 탐색': 452,
 '너비 우선 탐색': 411,
 '누적 합': 392,
 '시뮬레이션': 358,
 '사칙연산': 352,
 '조합론': 349,
 '해 구성하기': 321,
 '깊이 우선 탐색': 305,
 '많은 조건 분기': 295,
 '비트마스킹': 267,
 '백트래킹': 232,
 '데이크스트라': 227,
 '최단 경로': 205,
 '해시를 사용한 집합과 맵': 205,
 '분할 정복': 185,
 '분리 집합': 183,
 '스위핑': 169,
 '소수 판정': 167,
 '우선순위 큐': 165,
 '트리에서의 다이나믹 프로그래밍': 165,
 '트리를 사용한 집합과 맵': 164,
 '분할 정복을 이용한 거듭제곱': 150,
 '게임 이론': 148,
 '두 포인터': 143,
 '매개 변수 탐색': 142,
 '파싱': 129,
 '스택': 128,
 '느리게 갱신되는 세그먼트 트리': 125,
 '최대 유량': 124,
 '에라토스테네스의 체': 114,
 '비트필드를 이용한 다이나믹 프로그래밍': 113,
 '오프라인 쿼리': 103,
 '재귀': 101,
 '배낭 문제': 96,
 '확률론': 94,
 '임의 정밀도 / 큰 수 연산': 91,
 '값 / 좌표 압축': 91,
 '유클리드 호제법': 89,
 '이분 매칭': 77,
 '최소 스패닝 트리': 74,
 '볼록 껍질': 73,
 '플로이드–워셜': 71,
 '강한 연결 요소': 70,
 '최소 공통 조상': 68,
 '선형대수학': 67,
 '위상 정렬': 65,

In [12]:
len(cleaned_data)

201

In [25]:
tier_dict = {
    'Not ratable':0,
    'Bronze V': 1,
    'Bronze IV': 2,
    'Bronze III': 3,
    'Bronze II': 4,
    'Bronze I': 5,
    'Silver V': 6,
    'Silver IV': 7,
    'Silver III': 8,
    'Silver II': 9,
    'Silver I': 10,
    'Gold V': 11,
    'Gold IV': 12,
    'Gold III': 13,
    'Gold II': 14,
    'Gold I': 15,
    'Platinum V': 16,
    'Platinum IV': 17,
    'Platinum III': 18,
    'Platinum II': 19,
    'Platinum I': 20,
    'Diamond V': 21,
    'Diamond IV': 22,
    'Diamond III': 23,
    'Diamond II': 24,
    'Diamond I': 25,
    'Ruby V': 26,
    'Ruby IV': 27,
    'Ruby III': 28,
    'Ruby II': 29,
    'Ruby I': 30,
    'Master':31
}

In [56]:
a_query = "SELECT problem_number, problem_tier, problem_algorithm FROM problem_information_table"
cursor.execute(a_query)
total_problem_list = cursor.fetchall()
total_problem_list

[(1000, 'Bronze V', "['수학', '구현', '사칙연산']"),
 (1001, 'Bronze V', "['수학', '구현', '사칙연산']"),
 (1002, 'Silver III', "['수학', '기하학', '많은 조건 분기']"),
 (1003, 'Silver III', "['다이나믹 프로그래밍']"),
 (1004, 'Silver III', "['수학', '기하학']"),
 (1005, 'Gold III', "['다이나믹 프로그래밍', '그래프 이론', '위상 정렬', '방향 비순환 그래프']"),
 (1006, 'Platinum III', "['다이나믹 프로그래밍']"),
 (1007, 'Gold II', "['수학', '브루트포스 알고리즘']"),
 (1008, 'Bronze V', "['수학', '구현', '사칙연산']"),
 (1009, 'Bronze II', "['수학', '구현']"),
 (1010, 'Silver V', "['수학', '다이나믹 프로그래밍', '조합론']"),
 (1011, 'Gold V', "['수학']"),
 (1012, 'Silver II', "['그래프 이론', '그래프 탐색', '너비 우선 탐색', '깊이 우선 탐색']"),
 (1013, 'Gold V', "['문자열', '정규 표현식']"),
 (1014,
  'Platinum IV',
  "['다이나믹 프로그래밍', '비트마스킹', '최대 유량', '비트필드를 이용한 다이나믹 프로그래밍']"),
 (1015, 'Silver IV', "['정렬']"),
 (1016, 'Gold I', "['수학', '정수론', '소수 판정', '에라토스테네스의 체']"),
 (1017, 'Platinum III', "['수학', '정수론', '소수 판정', '에라토스테네스의 체', '이분 매칭']"),
 (1018, 'Silver IV', "['브루트포스 알고리즘']"),
 (1019, 'Gold I', "['수학']"),
 (1020, 'Platinum II',

In [32]:
import ast
user_id = '0_o'
a_query = "SELECT user_solved_problem_list, user_tier, user_failed_problem_list, user_weak_algorithm FROM user_information_table WHERE user_id = %s"
cursor.execute(a_query, ('0_o',))
user_info = cursor.fetchall()
user_solved_problem_list = ast.literal_eval(user_info[0][0])
user_tier = tier_dict[user_info[0][1]]
user_failed_problem_list = ast.literal_eval(user_info[0][2])
user_weak_algorithm = ast.literal_eval(user_info[0][3])


In [20]:
len(total_problem_list)

9029

In [17]:
len(user_solved_problem_list)

542

In [49]:
user_non_solved_problem = [data for data in total_problem_list if data[0] not in user_solved_problem_list]
user_non_solved_problem

[(1004, 'Silver III', "['수학', '기하학']"),
 (1006, 'Platinum III', "['다이나믹 프로그래밍']"),
 (1007, 'Gold II', "['수학', '브루트포스 알고리즘']"),
 (1013, 'Gold V', "['문자열', '정규 표현식']"),
 (1014,
  'Platinum IV',
  "['다이나믹 프로그래밍', '비트마스킹', '최대 유량', '비트필드를 이용한 다이나믹 프로그래밍']"),
 (1015, 'Silver IV', "['정렬']"),
 (1016, 'Gold I', "['수학', '정수론', '소수 판정', '에라토스테네스의 체']"),
 (1017, 'Platinum III', "['수학', '정수론', '소수 판정', '에라토스테네스의 체', '이분 매칭']"),
 (1019, 'Gold I', "['수학']"),
 (1020, 'Platinum II', "['다이나믹 프로그래밍']"),
 (1021, 'Silver III', "['자료 구조', '덱']"),
 (1022, 'Gold III', "['수학', '구현']"),
 (1023, 'Platinum III', "['수학', '다이나믹 프로그래밍', '조합론']"),
 (1024, 'Silver II', "['수학']"),
 (1025, 'Gold V', "['브루트포스 알고리즘']"),
 (1027, 'Gold IV', "['수학', '브루트포스 알고리즘', '기하학']"),
 (1028, 'Platinum V', "['다이나믹 프로그래밍', '누적 합']"),
 (1029, 'Gold I', "['다이나믹 프로그래밍', '비트마스킹', '비트필드를 이용한 다이나믹 프로그래밍']"),
 (1030, 'Gold III', "['구현', '분할 정복', '재귀']"),
 (1031, 'Diamond V', "['최대 유량']"),
 (1032, 'Bronze I', "['구현', '문자열']"),
 (1033, 'Gold II'

In [53]:
non_solved_problem_length = len(user_non_solved_problem)
user_libffm_df = pd.DataFrame(zip([user_id]*non_solved_problem_length, 
                                  [int(problem_number[0]) for problem_number in user_non_solved_problem], 
                                  [user_tier]*non_solved_problem_length, 
                                  [user_failed_problem_list] * non_solved_problem_length,
                                  [user_weak_algorithm[:5]] * non_solved_problem_length,
                                  [tier_dict[problem_tier[1]] for problem_tier in user_non_solved_problem],
                                  [ast.literal_eval(problem_algorithm[2]) for problem_algorithm in user_non_solved_problem]),
                                  columns = ['user_id', 'problem_number', 'user_tier', 'user_failed_problem_list', 'user_weak_algorithm', 'problem_tier', 'problem_algorithm'])

In [54]:
user_libffm_df

,user_id,problem_number,user_tier,user_failed_problem_list,user_weak_algorithm,problem_tier,problem_algorithm
0,0_o,1004,17,"[1389, 2496, 2517, 3197, 7578, 9328, 10165, 13...","[수학, 구현, 다이나믹 프로그래밍, 자료 구조, 그래프 이론]",8,"[수학, 기하학]"
1,0_o,1006,17,"[1389, 2496, 2517, 3197, 7578, 9328, 10165, 13...","[수학, 구현, 다이나믹 프로그래밍, 자료 구조, 그래프 이론]",18,[다이나믹 프로그래밍]
2,0_o,1007,17,"[1389, 2496, 2517, 3197, 7578, 9328, 10165, 13...","[수학, 구현, 다이나믹 프로그래밍, 자료 구조, 그래프 이론]",14,"[수학, 브루트포스 알고리즘]"
3,0_o,1013,17,"[1389, 2496, 2517, 3197, 7578, 9328, 10165, 13...","[수학, 구현, 다이나믹 프로그래밍, 자료 구조, 그래프 이론]",11,"[문자열, 정규 표현식]"
4,0_o,1014,17,"[1389, 2496, 2517, 3197, 7578, 9328, 10165, 13...","[수학, 구현, 다이나믹 프로그래밍, 자료 구조, 그래프 이론]",17,"[다이나믹 프로그래밍, 비트마스킹, 최대 유량, 비트필드를 이용한 다이나믹 프로그래밍]"
...,...,...,...,...,...,...,...
8482,0_o,31266,17,"[1389, 2496, 2517, 3197, 7578, 9328, 10165, 13...","[수학, 구현, 다이나믹 프로그래밍, 자료 구조, 그래프 이론]",16,"[다이나믹 프로그래밍, 그리디 알고리즘, 브루트포스 알고리즘, 정렬, 비트마스킹, ..."
8483,0_o,31267,17,"[1389, 2496, 2517, 3197, 7578, 9328, 10165, 13...","[수학, 구현, 다이나믹 프로그래밍, 자료 구조, 그래프 이론]",18,"[그리디 알고리즘, 이분 탐색, 매개 변수 탐색]"
8484,0_o,31268,17,"[1389, 2496, 2517, 3197, 7578, 9328, 10165, 13...","[수학, 구현, 다이나믹 프로그래밍, 자료 구조, 그래프 이론]",18,"[그래프 이론, 데이크스트라, 최단 경로, 오프라인 쿼리, 플로이드–워셜]"
8485,0_o,31283,17,"[1389, 2496, 2517, 3197, 7578, 9328, 10165, 13...","[수학, 구현, 다이나믹 프로그래밍, 자료 구조, 그래프 이론]",26,"[자료 구조, 기하학, 트리, 세그먼트 트리, 이분 탐색, 우선순위 큐, 작은 집합..."


In [57]:
from sklearn.preprocessing import LabelEncoder
import joblib

encoder_data_path = '../DataSet/Encoder/'

user_ID_Encoder = joblib.load(os.path.join(encoder_data_path, 'user_ID_Encoder.joblib'))
problem_number_Encoder = joblib.load(os.path.join(encoder_data_path, 'problem_number_Encoder.joblib'))
problem_algorithm_Encoder = joblib.load(os.path.join(encoder_data_path, 'problem_algorithm_Encoder.joblib'))

with open(os.path.join(encoder_data_path, 'tier_dict.pkl'), 'rb') as f:
    tier_dict = pickle.load(f)

c:\Users\win10KDH\anaconda3\lib\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator LabelEncoder from version 1.2.1 when using version 1.1.3. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [59]:
user_libffm_df['user_id'] = user_ID_Encoder.transform(user_libffm_df['user_id'])
user_libffm_df['problem_number'] = problem_number_Encoder.transform(user_libffm_df['problem_number'])
user_libffm_df['problem_algorithm'] = user_libffm_df['problem_algorithm'].apply(lambda x: problem_algorithm_Encoder.transform(x))
user_libffm_df['user_failed_problem_list'] = user_libffm_df['user_failed_problem_list'].apply(lambda x: problem_number_Encoder.transform(x))
user_libffm_df['user_weak_algorithm'] = user_libffm_df['user_weak_algorithm'].apply(lambda x: problem_algorithm_Encoder.transform(x))

In [60]:
user_libffm_df

,user_id,problem_number,user_tier,user_failed_problem_list,user_weak_algorithm,problem_tier,problem_algorithm
0,31,4,17,"[372, 1350, 1370, 1950, 2698, 2916, 3100, 4277...","[100, 17, 29, 137, 18]",8,"[100, 23]"
1,31,6,17,"[372, 1350, 1370, 1950, 2698, 2916, 3100, 4277...","[100, 17, 29, 137, 18]",18,[29]
2,31,7,17,"[372, 1350, 1370, 1950, 2698, 2916, 3100, 4277...","[100, 17, 29, 137, 18]",14,"[100, 82]"
3,31,13,17,"[372, 1350, 1370, 1950, 2698, 2916, 3100, 4277...","[100, 17, 29, 137, 18]",11,"[59, 143]"
4,31,14,17,"[372, 1350, 1370, 1950, 2698, 2916, 3100, 4277...","[100, 17, 29, 137, 18]",17,"[29, 85, 154, 86]"
...,...,...,...,...,...,...,...
8482,31,9024,17,"[372, 1350, 1370, 1950, 2698, 2916, 3100, 4277...","[100, 17, 29, 137, 18]",16,"[29, 20, 82, 144, 85, 86]"
8483,31,9025,17,"[372, 1350, 1370, 1950, 2698, 2916, 3100, 4277...","[100, 17, 29, 137, 18]",18,"[20, 131, 52]"
8484,31,9026,17,"[372, 1350, 1370, 1950, 2698, 2916, 3100, 4277...","[100, 17, 29, 137, 18]",18,"[18, 36, 153, 123, 182]"
8485,31,9027,17,"[372, 1350, 1370, 1950, 2698, 2916, 3100, 4277...","[100, 17, 29, 137, 18]",26,"[137, 23, 169, 95, 131, 125, 139]"


In [2]:
from urllib.parse import quote

user_id = "03mun"
wanted_algorithm_list = "[\"수학\"]"

# 리스트를 URL-encoded 문자열로 변환
encoded_wanted_algorithm_list = quote(str(wanted_algorithm_list))

# URL 조립
url = f"http://127.0.0.1:8000/get_list?user_id={user_id}&wanted_algorithm_list={encoded_wanted_algorithm_list}"

print(url)

http://127.0.0.1:8000/get_list?user_id=03mun&wanted_algorithm_list=%5B%22%EC%88%98%ED%95%99%22%5D


In [5]:
import pandas as pd
import numpy as np
import time
import os
import urllib.request
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
import pickle
import re
from sklearn.preprocessing import LabelEncoder
import joblib
from collections import Counter
from collections import defaultdict
import ast
import mysql.connector

from selenium.common.exceptions import StaleElementReferenceException
from selenium.common.exceptions import NoSuchElementException

chrome_options = Options()
chrome_options.add_argument('--headless')  # 웹 브라우저를 표시하지 않고 백그라운드에서 실행
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

In [6]:
from sklearn.preprocessing import LabelEncoder
import joblib

encoder_data_path = '../DataSet/Encoder/'

user_ID_Encoder = joblib.load(os.path.join(encoder_data_path, 'user_ID_Encoder.joblib'))
problem_number_Encoder = joblib.load(os.path.join(encoder_data_path, 'problem_number_Encoder.joblib'))
problem_algorithm_Encoder = joblib.load(os.path.join(encoder_data_path, 'problem_algorithm_Encoder.joblib'))

c:\Users\win10KDH\anaconda3\lib\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator LabelEncoder from version 1.2.1 when using version 1.1.3. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [7]:
user_ID_Encoder

LabelEncoder()

In [8]:
encoded_classes = dict(zip(user_ID_Encoder.classes_, user_ID_Encoder.transform(user_ID_Encoder.classes_)))

In [9]:
encoded_classes

{'01026488131': 0,
 '0106658lee': 1,
 '01089292615': 2,
 '01234': 3,
 '0123ox': 4,
 '01jm': 5,
 '0203peter': 6,
 '0214kbs': 7,
 '021gundam': 8,
 '0225az': 9,
 '02kjw0203': 10,
 '0309mun': 11,
 '0318asd': 12,
 '0321minji': 13,
 '0326bsj': 14,
 '0512tini': 15,
 '051ssaa': 16,
 '06090419': 17,
 '0613antonio': 18,
 '06elvalove': 19,
 '06zzkimzz06': 20,
 '0708jyc': 21,
 '072756': 22,
 '07yoon13': 23,
 '0909oje': 24,
 '0911bim': 25,
 '0913vision': 26,
 '0917ba': 27,
 '0923kdh': 28,
 '0929': 29,
 '0930dhs': 30,
 '0_o': 31,
 '0at_x': 32,
 '0bliviate': 33,
 '0cookieboy0': 34,
 '0do': 35,
 '0general': 36,
 '0gyeong99': 37,
 '0h328': 38,
 '0nrich': 39,
 '0onionion0': 40,
 '0pao0': 41,
 '0rch1d': 42,
 '0sthcp': 43,
 '0styycp': 44,
 '0xbc614e': 45,
 '0xchaser': 46,
 '0xe82de': 47,
 '1000chw': 48,
 '1000sxbh': 49,
 '1004gite': 50,
 '100_101_cpp': 51,
 '100coding': 52,
 '100points': 53,
 '1017jiy': 54,
 '1031nice': 55,
 '107esther': 56,
 '10jobss': 57,
 '110000110101': 58,
 '1114nhj': 59,
 '1119wj': 

In [10]:
len(encoded_classes)

15416

In [3]:
import mysql.connector

conn = mysql.connector.connect(
    user='root',
    password='llsy13579!',
    host='localhost',
    port=3306,
    database='recommend_project'
)

DB_table_name = 'recommend_project'

cursor = conn.cursor()
use_query = f"USE {DB_table_name}"

cursor.execute(use_query)

set_query = "SET innodb_lock_wait_timeout = 28800;"

cursor.execute(set_query)

In [16]:
a_query = "UPDATE user_information_table SET user_key = %s WHERE user_id = %s;"

for k,v in encoded_classes.items():
    cursor.execute(a_query, (int(v), k))

conn.commit()

In [18]:
encoded_classes = dict(zip(problem_number_Encoder.classes_, problem_number_Encoder.transform(problem_number_Encoder.classes_)))
encoded_classes

{1000: 0,
 1001: 1,
 1002: 2,
 1003: 3,
 1004: 4,
 1005: 5,
 1006: 6,
 1007: 7,
 1008: 8,
 1009: 9,
 1010: 10,
 1011: 11,
 1012: 12,
 1013: 13,
 1014: 14,
 1015: 15,
 1016: 16,
 1017: 17,
 1018: 18,
 1019: 19,
 1020: 20,
 1021: 21,
 1022: 22,
 1023: 23,
 1024: 24,
 1025: 25,
 1026: 26,
 1027: 27,
 1028: 28,
 1029: 29,
 1030: 30,
 1031: 31,
 1032: 32,
 1033: 33,
 1034: 34,
 1035: 35,
 1036: 36,
 1037: 37,
 1038: 38,
 1039: 39,
 1040: 40,
 1041: 41,
 1042: 42,
 1043: 43,
 1044: 44,
 1045: 45,
 1046: 46,
 1047: 47,
 1048: 48,
 1049: 49,
 1050: 50,
 1051: 51,
 1052: 52,
 1053: 53,
 1054: 54,
 1055: 55,
 1056: 56,
 1057: 57,
 1058: 58,
 1059: 59,
 1060: 60,
 1061: 61,
 1062: 62,
 1063: 63,
 1064: 64,
 1065: 65,
 1066: 66,
 1067: 67,
 1068: 68,
 1069: 69,
 1070: 70,
 1071: 71,
 1072: 72,
 1073: 73,
 1074: 74,
 1075: 75,
 1076: 76,
 1077: 77,
 1078: 78,
 1079: 79,
 1080: 80,
 1081: 81,
 1082: 82,
 1083: 83,
 1084: 84,
 1085: 85,
 1086: 86,
 1087: 87,
 1088: 88,
 1089: 89,
 1090: 90,
 1091: 91

In [23]:
a_query = "UPDATE problem_information_table SET problem_key = %s WHERE problem_number = %s;"

for k,v in encoded_classes.items():
    cursor.execute(a_query, (int(v), int(k)))

conn.commit()

In [25]:
create_problem_algorithm_table_query = "CREATE TABLE IF NOT EXISTS problem_algorithm_encoder_table ( \
    problem_algorithm_number INT PRIMARY KEY, \
    problem_algorithm VARCHAR(1000) \
);"
cursor.execute(create_problem_algorithm_table_query)
conn.commit()

In [26]:
encoded_classes = dict(zip(problem_algorithm_Encoder.classes_, problem_algorithm_Encoder.transform(problem_algorithm_Encoder.classes_)))
encoded_classes

{'': 0,
 '0-1 너비 우선 탐색': 1,
 '2-sat': 2,
 '3차원 기하학': 3,
 '4차원 이상의 기하학': 4,
 'aliens 트릭': 5,
 'heavy-light 분할': 6,
 'kmp': 7,
 "mo's": 8,
 'utf-8 입력 처리': 9,
 'z': 10,
 '가우스 소거법': 11,
 '가장 긴 증가하는 부분 수열: o(n log n)': 12,
 '값 / 좌표 압축': 13,
 '강한 연결 요소': 14,
 '게임 이론': 15,
 '고속 푸리에 변환': 16,
 '구현': 17,
 '그래프 이론': 18,
 '그래프 탐색': 19,
 '그리디 알고리즘': 20,
 '그린 정리': 21,
 '기댓값의 선형성': 22,
 '기하학': 23,
 '깊이 우선 탐색': 24,
 '너비 우선 탐색': 25,
 '누적 합': 26,
 '느리게 갱신되는 세그먼트 트리': 27,
 '다각형의 넓이': 28,
 '다이나믹 프로그래밍': 29,
 '다중 대입값 계산': 30,
 '다차원 세그먼트 트리': 31,
 '다항식 보간법': 32,
 '단절점과 단절선': 33,
 '단조 큐를 이용한 최적화': 34,
 '담금질 기법': 35,
 '데이크스트라': 36,
 '데카르트 트리': 37,
 '덱': 38,
 '덱을 이용한 구간 최댓값 트릭': 39,
 '덱을 이용한 다이나믹 프로그래밍': 40,
 '델로네 삼각분할': 41,
 '도미네이터 트리': 42,
 '도형에서의 불 연산': 43,
 '두 포인터': 44,
 '라빈–카프': 45,
 '런타임 전의 전처리': 46,
 '로프': 47,
 '뤼카 정리': 48,
 '린드스트롬–게셀–비엔노 보조정리': 49,
 '링크/컷 트리': 50,
 '많은 조건 분기': 51,
 '매개 변수 탐색': 52,
 '매내처': 53,
 '매트로이드': 54,
 '머지 소트 트리': 55,
 '모듈로 곱셈 역원': 56,
 '뫼비우스 반전 공식': 57,
 '무작위화': 58,
 '문자열': 59,
 

In [27]:
a_query = "INSERT INTO problem_algorithm_encoder_table (problem_algorithm_number, problem_algorithm) VALUES (%s, %s)"

for k,v in encoded_classes.items():
    cursor.execute(a_query, (int(v), k))

conn.commit()

In [28]:
tier_dict

NameError: name 'tier_dict' is not defined

In [29]:
tier_dict = {
    'Not ratable':0,
    'Bronze V': 1,
    'Bronze IV': 2,
    'Bronze III': 3,
    'Bronze II': 4,
    'Bronze I': 5,
    'Silver V': 6,
    'Silver IV': 7,
    'Silver III': 8,
    'Silver II': 9,
    'Silver I': 10,
    'Gold V': 11,
    'Gold IV': 12,
    'Gold III': 13,
    'Gold II': 14,
    'Gold I': 15,
    'Platinum V': 16,
    'Platinum IV': 17,
    'Platinum III': 18,
    'Platinum II': 19,
    'Platinum I': 20,
    'Diamond V': 21,
    'Diamond IV': 22,
    'Diamond III': 23,
    'Diamond II': 24,
    'Diamond I': 25,
    'Ruby V': 26,
    'Ruby IV': 27,
    'Ruby III': 28,
    'Ruby II': 29,
    'Ruby I': 30,
    'Master':31
}

In [17]:
import re

input_string = "수학,다이나믹 프로그래밍,깊이 우선 탐색"

a = input_string.split(',')
a

['수학', '다이나믹 프로그래밍', '깊이 우선 탐색']

In [30]:

create_tier_encoder_table_query = "CREATE TABLE IF NOT EXISTS tier_encoder_table ( \
    tier_number INT PRIMARY KEY, \
    tier VARCHAR(30) \
);"

cursor.execute(create_tier_encoder_table_query)
conn.commit()

In [31]:
a_query = "INSERT INTO tier_encoder_table (tier_number, tier) VALUES (%s, %s)"

for k,v in tier_dict.items():
    cursor.execute(a_query, (int(v), k))

conn.commit()

In [3]:
a = "SELECT max(user_key) FROM user_information_table"
cursor.execute(a)
c = cursor.fetchall()
b = c[0][0] + 1

In [4]:
b

15416

In [46]:
create_tier_encoder_table_query = "ALTER TABLE user_information_table ADD COLUMN experiment INT UNIQUE NOT NULL auto_increment;"
cursor.execute(create_tier_encoder_table_query)
conn.commit()

In [44]:
import random
s = [i for i in range(0,32)]
random.shuffle(s)
s

[29,
 15,
 7,
 3,
 6,
 30,
 13,
 4,
 8,
 14,
 0,
 31,
 9,
 26,
 5,
 19,
 16,
 21,
 27,
 10,
 17,
 11,
 25,
 20,
 18,
 28,
 23,
 2,
 22,
 12,
 1,
 24]

In [45]:

a_query = "INSERT INTO experiment (tier_number, tier) VALUES (%s, %s)"


for k,v in tier_dict.items():
    cursor.execute(a_query, (s[int(v)], k))

conn.commit()

In [ ]:
a_query = "INSERT INTO experiment (tier_number, tier) VALUES (%s, %s)"

In [60]:
query = "SELECT * FROM user_problem_interaction_table"

cursor.execute(query)
user_problem_interaction = cursor.fetchall()

user_problem_interaction = pd.DataFrame(user_problem_interaction, columns = ['user_problem_key',
'user_id',
'problem_number',
'correct_attempts_count',
'incorrect_attempts_count',
'time_limit_exceeded_count',
'memory_limit_exceeded_count',
'output_limit_exceeded_count',
'runtime_error_count', 
'compile_error_count',
'output_format_issue_count', 
'most_used_language',
'best_memory_usage',
'fastest_execution_time',
'memory_limit_optimal_solution_submission_number', 
'time_limit_optimal_solution'])

query = "SELECT * FROM user_information_table"

cursor.execute(query)
user_information = pd.DataFrame(cursor.fetchall(), columns = ['user_id', 
    'user_key',                                                         
    'user_tier',
    'user_rank',
    'user_solved_problem_list',
    'user_failed_problem_list',
    'user_solved_problem_count',
    'user_weak_algorithm'])

query = "SELECT * FROM problem_information_table"

cursor.execute(query)
problem_information = pd.DataFrame(cursor.fetchall(), columns = ['problem_number',
'problem_key',
'problem_name',
'problem_tier',
'problem_algorithm',
'problem_time_condition',
'problem_memory_condition',
'problem_submission_count',
'problem_answer_submission_count',
'problem_answered_people_count',
'problem_correct_rate',
'problem_description',
'problem_input',
'problem_output',
'other_result_form_1',
'other_result_form_2'])

new_columns = {'user_key': 'user_id', 'problem_key': 'problem_number'}

libffm_df = pd.DataFrame(user_problem_interaction.iloc[:,1:11])
libffm_df.insert(0, 'target', libffm_df.iloc[:,3:10].sum(axis=1))
libffm_df.drop(['correct_attempts_count', 'incorrect_attempts_count', 'time_limit_exceeded_count', 'memory_limit_exceeded_count', 'output_limit_exceeded_count', 'runtime_error_count', 'compile_error_count', 'output_format_issue_count'], axis=1, inplace=True)
libffm_df = pd.merge(libffm_df, user_information[['user_id', 'user_key','user_tier','user_failed_problem_list','user_weak_algorithm']], how='inner', on = 'user_id')
libffm_df = pd.merge(libffm_df, problem_information[['problem_number', 'problem_key','problem_tier','problem_algorithm']], how='inner', on = 'problem_number')
libffm_df.drop(['user_id','problem_number'], axis=1, inplace = True)
libffm_df.rename(columns=new_columns, inplace=True)
libffm_df = libffm_df.sort_values(by = 'user_id').reset_index(drop=True)

In [47]:
libffm_df

,target,user_id,user_tier,user_failed_problem_list,user_weak_algorithm,problem_number,problem_tier,problem_algorithm
0,0,0,Gold I,"[1059, 1300, 1504, 2502, 2531, 2637, 2725, 606...","['그래프 이론', '그래프 탐색', '너비 우선 탐색', '구현', '브루트포스 ...",697,Silver IV,"['수학', '구현']"
1,4,0,Gold I,"[1059, 1300, 1504, 2502, 2531, 2637, 2725, 606...","['그래프 이론', '그래프 탐색', '너비 우선 탐색', '구현', '브루트포스 ...",167,Gold II,"['그래프 이론', '그래프 탐색', '트리', '깊이 우선 탐색']"
2,1,0,Gold I,"[1059, 1300, 1504, 2502, 2531, 2637, 2725, 606...","['그래프 이론', '그래프 탐색', '너비 우선 탐색', '구현', '브루트포스 ...",3574,Silver IV,"['문자열', '정렬']"
3,0,0,Gold I,"[1059, 1300, 1504, 2502, 2531, 2637, 2725, 606...","['그래프 이론', '그래프 탐색', '너비 우선 탐색', '구현', '브루트포스 ...",3538,Gold IV,"['구현', '그래프 이론', '그래프 탐색', '시뮬레이션', '너비 우선 탐색']"
4,0,0,Gold I,"[1059, 1300, 1504, 2502, 2531, 2637, 2725, 606...","['그래프 이론', '그래프 탐색', '너비 우선 탐색', '구현', '브루트포스 ...",4436,Gold IV,"['구현', '그래프 이론', '브루트포스 알고리즘', '그래프 탐색', '너비 우..."
...,...,...,...,...,...,...,...,...
5046140,0,15415,Gold I,"[11000, 11049]","['그래프 이론', '자료 구조', '그래프 탐색', '구현', '다이나믹 프로그래...",717,Silver V,"['수학', '구현', '문자열']"
5046141,2,15415,Gold I,"[11000, 11049]","['그래프 이론', '자료 구조', '그래프 탐색', '구현', '다이나믹 프로그래...",1464,Silver I,"['구현', '브루트포스 알고리즘']"
5046142,0,15415,Gold I,"[11000, 11049]","['그래프 이론', '자료 구조', '그래프 탐색', '구현', '다이나믹 프로그래...",1674,Silver III,"['구현', '문자열']"
5046143,0,15415,Gold I,"[11000, 11049]","['그래프 이론', '자료 구조', '그래프 탐색', '구현', '다이나믹 프로그래...",2863,Gold V,"['그래프 이론', '그래프 탐색', '너비 우선 탐색']"


In [62]:
libffm_df = libffm_df.iloc[:10000,:]

In [49]:
libffm_df

,target,user_id,user_tier,user_failed_problem_list,user_weak_algorithm,problem_number,problem_tier,problem_algorithm
0,0,0,Gold I,"[1059, 1300, 1504, 2502, 2531, 2637, 2725, 606...","['그래프 이론', '그래프 탐색', '너비 우선 탐색', '구현', '브루트포스 ...",697,Silver IV,"['수학', '구현']"
1,4,0,Gold I,"[1059, 1300, 1504, 2502, 2531, 2637, 2725, 606...","['그래프 이론', '그래프 탐색', '너비 우선 탐색', '구현', '브루트포스 ...",167,Gold II,"['그래프 이론', '그래프 탐색', '트리', '깊이 우선 탐색']"
2,1,0,Gold I,"[1059, 1300, 1504, 2502, 2531, 2637, 2725, 606...","['그래프 이론', '그래프 탐색', '너비 우선 탐색', '구현', '브루트포스 ...",3574,Silver IV,"['문자열', '정렬']"
3,0,0,Gold I,"[1059, 1300, 1504, 2502, 2531, 2637, 2725, 606...","['그래프 이론', '그래프 탐색', '너비 우선 탐색', '구현', '브루트포스 ...",3538,Gold IV,"['구현', '그래프 이론', '그래프 탐색', '시뮬레이션', '너비 우선 탐색']"
4,0,0,Gold I,"[1059, 1300, 1504, 2502, 2531, 2637, 2725, 606...","['그래프 이론', '그래프 탐색', '너비 우선 탐색', '구현', '브루트포스 ...",4436,Gold IV,"['구현', '그래프 이론', '브루트포스 알고리즘', '그래프 탐색', '너비 우..."
...,...,...,...,...,...,...,...,...
9995,3,30,Gold I,"[1003, 1024, 1038, 1062, 1120, 1152, 1175, 125...","['그래프 이론', '그래프 탐색', '구현', '너비 우선 탐색', '수학', '...",4274,Gold IV,"['그래프 이론', '그래프 탐색', '너비 우선 탐색']"
9996,0,30,Gold I,"[1003, 1024, 1038, 1062, 1120, 1152, 1175, 125...","['그래프 이론', '그래프 탐색', '구현', '너비 우선 탐색', '수학', '...",1575,Bronze IV,['구현']
9997,2,30,Gold I,"[1003, 1024, 1038, 1062, 1120, 1152, 1175, 125...","['그래프 이론', '그래프 탐색', '구현', '너비 우선 탐색', '수학', '...",1576,Bronze V,"['구현', '문자열']"
9998,2,30,Gold I,"[1003, 1024, 1038, 1062, 1120, 1152, 1175, 125...","['그래프 이론', '그래프 탐색', '구현', '너비 우선 탐색', '수학', '...",664,Bronze II,"['수학', '사칙연산']"


In [50]:
problem_algorithm_encoder_query = 'SELECT problem_algorithm, problem_algorithm_number FROM problem_algorithm_encoder_table'
cursor.execute(problem_algorithm_encoder_query)
problem_algorithm_encoder = cursor.fetchall()

In [25]:
problem_algorithm_encoder

[('', 0),
 ('0-1 너비 우선 탐색', 1),
 ('2-sat', 2),
 ('3차원 기하학', 3),
 ('4차원 이상의 기하학', 4),
 ('aliens 트릭', 5),
 ('heavy-light 분할', 6),
 ('kmp', 7),
 ("mo's", 8),
 ('utf-8 입력 처리', 9),
 ('z', 10),
 ('가우스 소거법', 11),
 ('가장 긴 증가하는 부분 수열: o(n log n)', 12),
 ('값 / 좌표 압축', 13),
 ('강한 연결 요소', 14),
 ('게임 이론', 15),
 ('고속 푸리에 변환', 16),
 ('구현', 17),
 ('그래프 이론', 18),
 ('그래프 탐색', 19),
 ('그리디 알고리즘', 20),
 ('그린 정리', 21),
 ('기댓값의 선형성', 22),
 ('기하학', 23),
 ('깊이 우선 탐색', 24),
 ('너비 우선 탐색', 25),
 ('누적 합', 26),
 ('느리게 갱신되는 세그먼트 트리', 27),
 ('다각형의 넓이', 28),
 ('다이나믹 프로그래밍', 29),
 ('다중 대입값 계산', 30),
 ('다차원 세그먼트 트리', 31),
 ('다항식 보간법', 32),
 ('단절점과 단절선', 33),
 ('단조 큐를 이용한 최적화', 34),
 ('담금질 기법', 35),
 ('데이크스트라', 36),
 ('데카르트 트리', 37),
 ('덱', 38),
 ('덱을 이용한 구간 최댓값 트릭', 39),
 ('덱을 이용한 다이나믹 프로그래밍', 40),
 ('델로네 삼각분할', 41),
 ('도미네이터 트리', 42),
 ('도형에서의 불 연산', 43),
 ('두 포인터', 44),
 ('라빈–카프', 45),
 ('런타임 전의 전처리', 46),
 ('로프', 47),
 ('뤼카 정리', 48),
 ('린드스트롬–게셀–비엔노 보조정리', 49),
 ('링크/컷 트리', 50),
 ('많은 조건 분기', 51),
 ('매개 변수 탐색', 52),


In [51]:
problem_algorithm_encoder = dict(problem_algorithm_encoder)

In [52]:
tier_encoder_query = 'SELECT tier, tier_number FROM tier_encoder_table'
cursor.execute(tier_encoder_query)
tier_encoder = cursor.fetchall()

In [53]:
tier_encoder

[('Not ratable', 0),
 ('Bronze V', 1),
 ('Bronze IV', 2),
 ('Bronze III', 3),
 ('Bronze II', 4),
 ('Bronze I', 5),
 ('Silver V', 6),
 ('Silver IV', 7),
 ('Silver III', 8),
 ('Silver II', 9),
 ('Silver I', 10),
 ('Gold V', 11),
 ('Gold IV', 12),
 ('Gold III', 13),
 ('Gold II', 14),
 ('Gold I', 15),
 ('Platinum V', 16),
 ('Platinum IV', 17),
 ('Platinum III', 18),
 ('Platinum II', 19),
 ('Platinum I', 20),
 ('Diamond V', 21),
 ('Diamond IV', 22),
 ('Diamond III', 23),
 ('Diamond II', 24),
 ('Diamond I', 25),
 ('Ruby V', 26),
 ('Ruby IV', 27),
 ('Ruby III', 28),
 ('Ruby II', 29),
 ('Ruby I', 30),
 ('Master', 31)]

In [54]:
tier_encoder = dict(tier_encoder)

In [55]:
problem_number_encoding_query = 'SELECT problem_number, problem_key FROM problem_information_table'
cursor.execute(problem_number_encoding_query)
problem_number_encoder = cursor.fetchall()

In [56]:
problem_number_encoder = dict(problem_number_encoder)

In [57]:
problem_number_encoder

{1000: 0,
 1001: 1,
 1002: 2,
 1003: 3,
 1004: 4,
 1005: 5,
 1006: 6,
 1007: 7,
 1008: 8,
 1009: 9,
 1010: 10,
 1011: 11,
 1012: 12,
 1013: 13,
 1014: 14,
 1015: 15,
 1016: 16,
 1017: 17,
 1018: 18,
 1019: 19,
 1020: 20,
 1021: 21,
 1022: 22,
 1023: 23,
 1024: 24,
 1025: 25,
 1026: 26,
 1027: 27,
 1028: 28,
 1029: 29,
 1030: 30,
 1031: 31,
 1032: 32,
 1033: 33,
 1034: 34,
 1035: 35,
 1036: 36,
 1037: 37,
 1038: 38,
 1039: 39,
 1040: 40,
 1041: 41,
 1042: 42,
 1043: 43,
 1044: 44,
 1045: 45,
 1046: 46,
 1047: 47,
 1048: 48,
 1049: 49,
 1050: 50,
 1051: 51,
 1052: 52,
 1053: 53,
 1054: 54,
 1055: 55,
 1056: 56,
 1057: 57,
 1058: 58,
 1059: 59,
 1060: 60,
 1061: 61,
 1062: 62,
 1063: 63,
 1064: 64,
 1065: 65,
 1066: 66,
 1067: 67,
 1068: 68,
 1069: 69,
 1070: 70,
 1071: 71,
 1072: 72,
 1073: 73,
 1074: 74,
 1075: 75,
 1076: 76,
 1077: 77,
 1078: 78,
 1079: 79,
 1080: 80,
 1081: 81,
 1082: 82,
 1083: 83,
 1084: 84,
 1085: 85,
 1086: 86,
 1087: 87,
 1088: 88,
 1089: 89,
 1090: 90,
 1091: 91

In [63]:
libffm_df['problem_algorithm'] = libffm_df['problem_algorithm'].apply(ast.literal_eval)
libffm_df['user_weak_algorithm'] = libffm_df['user_weak_algorithm'].apply(ast.literal_eval)
libffm_df['user_failed_problem_list'] = libffm_df['user_failed_problem_list'].apply(ast.literal_eval)
libffm_df['problem_algorithm'] = libffm_df['problem_algorithm'].apply(lambda x: list(map(problem_algorithm_encoder.get, x)))
libffm_df['user_weak_algorithm'] = libffm_df['user_weak_algorithm'].apply(lambda x: list(map(problem_algorithm_encoder.get, x)))
libffm_df['user_failed_problem_list'] = libffm_df['user_failed_problem_list'].apply(lambda x: list(map(problem_number_encoder.get, x)))
libffm_df['user_tier'] = libffm_df['user_tier'].map(tier_encoder)
libffm_df['problem_tier'] = libffm_df['problem_tier'].map(tier_encoder)

In [64]:
libffm_df

,target,user_id,user_tier,user_failed_problem_list,user_weak_algorithm,problem_number,problem_tier,problem_algorithm
0,0,0,15,"[59, 294, 474, 1356, 1383, 1485, 1561, 2561, 2...","[18, 19, 25, 17, 82, 108, 100, 29, 137, 24]",697,7,"[100, 17]"
1,4,0,15,"[59, 294, 474, 1356, 1383, 1485, 1561, 2561, 2...","[18, 19, 25, 17, 82, 108, 100, 29, 137, 24]",167,14,"[18, 19, 169, 24]"
2,1,0,15,"[59, 294, 474, 1356, 1383, 1485, 1561, 2561, 2...","[18, 19, 25, 17, 82, 108, 100, 29, 137, 24]",3574,7,"[59, 144]"
3,0,0,15,"[59, 294, 474, 1356, 1383, 1485, 1561, 2561, 2...","[18, 19, 25, 17, 82, 108, 100, 29, 137, 24]",3538,12,"[17, 18, 19, 108, 25]"
4,0,0,15,"[59, 294, 474, 1356, 1383, 1485, 1561, 2561, 2...","[18, 19, 25, 17, 82, 108, 100, 29, 137, 24]",4436,12,"[17, 18, 82, 19, 25]"
...,...,...,...,...,...,...,...,...
9995,3,30,15,"[3, 24, 38, 62, 120, 152, 175, 251, 330, 333, ...","[18, 19, 17, 25, 100, 24, 82, 59, 144, 66]",4274,12,"[18, 19, 25]"
9996,0,30,15,"[3, 24, 38, 62, 120, 152, 175, 251, 330, 333, ...","[18, 19, 17, 25, 100, 24, 82, 59, 144, 66]",1575,2,[17]
9997,2,30,15,"[3, 24, 38, 62, 120, 152, 175, 251, 330, 333, ...","[18, 19, 17, 25, 100, 24, 82, 59, 144, 66]",1576,1,"[17, 59]"
9998,2,30,15,"[3, 24, 38, 62, 120, 152, 175, 251, 330, 333, ...","[18, 19, 17, 25, 100, 24, 82, 59, 144, 66]",664,4,"[100, 87]"


In [19]:

problem_other_result_form_query = f'SELECT problem_number, other_result_form_1, other_result_form_2 FROM problem_information_table'

cursor.execute(problem_other_result_form_query)
problem_other_result_form = [(i[0],i[1],i[2]) for i in cursor.fetchall()]
problem_other_result_form_list = []


for p_info in problem_other_result_form:

    p_num, form_1, form_2 = p_info
    
    if(form_1 != 0 or form_2 != 0):
        problem_other_result_form_list.append(p_num)

problem_other_result_form_list

[2545,
 3090,
 4149,
 5522,
 5523,
 5524,
 5525,
 5812,
 5814,
 5819,
 5820,
 5821,
 5822,
 5823,
 8872,
 8875,
 8979,
 8980,
 8981,
 8982,
 8983,
 8984,
 8985,
 8986,
 8987,
 10070,
 10072,
 10073,
 10074,
 10075,
 10093,
 10096,
 10162,
 10163,
 10164,
 10165,
 10166,
 10167,
 10168,
 10169,
 10227,
 10713,
 10714,
 10715,
 10717,
 10755,
 10756,
 10833,
 10834,
 10835,
 10836,
 10837,
 10840,
 10841,
 10842,
 10919,
 10922,
 10923,
 10924,
 11933,
 11941,
 11985,
 12095,
 12752,
 13300,
 13301,
 13302,
 13303,
 13304,
 13305,
 13306,
 13308,
 13309,
 13310,
 14696,
 14697,
 14863,
 14864,
 14865,
 14866,
 14867,
 14868,
 14869,
 14870,
 15553,
 15554,
 15624,
 15704,
 15740,
 15769,
 15770,
 15772,
 15773,
 15775,
 15776,
 15777,
 15778,
 15779,
 15792,
 15820,
 15821,
 15822,
 15823,
 15824,
 15825,
 15826,
 15827,
 15828,
 15829,
 15830,
 15831,
 15933,
 15934,
 15964,
 15965,
 15966,
 15967,
 15968,
 15969,
 15970,
 15971,
 15972,
 15973,
 15974,
 15975,
 15976,
 15977,
 15979,
 

In [22]:
type(problem_other_result_form_list[0])

int

In [20]:
len(problem_other_result_form_list)

452

In [6]:
len(problem_other_result_form_dict_1)

{2545: 3,
 3090: 100,
 5522: 100,
 5523: 100,
 5524: 100,
 5525: 100,
 5812: 100,
 5814: 100,
 5819: 100,
 5820: 100,
 5821: 100,
 5822: 100,
 5823: 100,
 8872: 100,
 8875: 100,
 8979: 100,
 8980: 100,
 8981: 100,
 8982: 100,
 8983: 100,
 8984: 100,
 8985: 100,
 8986: 100,
 8987: 100,
 10070: 100,
 10072: 20,
 10073: 35,
 10074: 45,
 10075: 100,
 10093: 100,
 10096: 100,
 10162: 100,
 10163: 100,
 10164: 100,
 10165: 100,
 10166: 100,
 10167: 100,
 10168: 100,
 10169: 100,
 10227: 100,
 10713: 100,
 10714: 100,
 10715: 100,
 10717: 100,
 10755: 100,
 10756: 100,
 10833: 100,
 10834: 100,
 10835: 100,
 10836: 100,
 10837: 100,
 10840: 100,
 10841: 100,
 10842: 100,
 10919: 100,
 10922: 100,
 10923: 54,
 10924: 46,
 11933: 100,
 11941: 100,
 11985: 100,
 12095: 100000,
 12752: 100,
 13300: 100,
 13301: 100,
 13302: 100,
 13303: 100,
 13304: 100,
 13305: 100,
 13306: 100,
 13308: 100,
 13309: 100,
 13310: 100,
 14696: 100,
 14697: 100,
 14863: 100,
 14864: 100,
 14865: 100,
 14866: 100,
 

In [9]:
len(problem_other_result_form_dict_1)

435

In [8]:
problem_other_result_form_dict_2

{4149: 161,
 15624: 201,
 20164: 102,
 20165: 11,
 20166: 16,
 20167: 39,
 20168: 18,
 20181: 66,
 20182: 16,
 20183: 43,
 21133: 500,
 21275: 24,
 21276: 19,
 21277: 19,
 21278: 19,
 21279: 16,
 24727: 100}

In [10]:
len(problem_other_result_form_dict_2)

17

In [23]:
problem_algorithm_list_query = f'SELECT problem_algorithm FROM problem_algorithm_encoder_table;'
cursor.execute(problem_algorithm_list_query)
problem_algorithm_list = cursor.fetchall()


In [26]:
problem_algorithm_list = [p[0] for p in problem_algorithm_list]

In [27]:
problem_algorithm_list

['',
 '0-1 너비 우선 탐색',
 '2-sat',
 '3차원 기하학',
 '4차원 이상의 기하학',
 'aliens 트릭',
 'heavy-light 분할',
 'kmp',
 "mo's",
 'utf-8 입력 처리',
 'z',
 '가우스 소거법',
 '가장 긴 증가하는 부분 수열: o(n log n)',
 '값 / 좌표 압축',
 '강한 연결 요소',
 '게임 이론',
 '고속 푸리에 변환',
 '구현',
 '그래프 이론',
 '그래프 탐색',
 '그리디 알고리즘',
 '그린 정리',
 '기댓값의 선형성',
 '기하학',
 '깊이 우선 탐색',
 '너비 우선 탐색',
 '누적 합',
 '느리게 갱신되는 세그먼트 트리',
 '다각형의 넓이',
 '다이나믹 프로그래밍',
 '다중 대입값 계산',
 '다차원 세그먼트 트리',
 '다항식 보간법',
 '단절점과 단절선',
 '단조 큐를 이용한 최적화',
 '담금질 기법',
 '데이크스트라',
 '데카르트 트리',
 '덱',
 '덱을 이용한 구간 최댓값 트릭',
 '덱을 이용한 다이나믹 프로그래밍',
 '델로네 삼각분할',
 '도미네이터 트리',
 '도형에서의 불 연산',
 '두 포인터',
 '라빈–카프',
 '런타임 전의 전처리',
 '로프',
 '뤼카 정리',
 '린드스트롬–게셀–비엔노 보조정리',
 '링크/컷 트리',
 '많은 조건 분기',
 '매개 변수 탐색',
 '매내처',
 '매트로이드',
 '머지 소트 트리',
 '모듈로 곱셈 역원',
 '뫼비우스 반전 공식',
 '무작위화',
 '문자열',
 '물리학',
 '미적분학',
 '밀러–라빈 소수 판별법',
 '반평면 교집합',
 '방향 비순환 그래프',
 '배낭 문제',
 '백트래킹',
 '번사이드 보조정리',
 '벌리캠프–매시',
 '베이즈 정리',
 '벨만–포드',
 '병렬 이분 탐색',
 '보로노이 다이어그램',
 '보이어–무어 다수결 투표',
 '볼록 껍질',
 '볼록 껍질을 이용한 최적화',
 '볼록 다각형 내부의 점 판정',
 '부분집합의 합

In [28]:
def get_problem_algorithm_list():

    problem_algorithm_list_query = f'SELECT problem_algorithm FROM problem_algorithm_encoder_table;'
    cursor.execute(problem_algorithm_list_query)
    problem_algorithm_list = cursor.fetchall()
    problem_algorithm_list = [p[0] for p in problem_algorithm_list]

    return problem_algorithm_list

In [36]:
wanted_algorithm_list = ['수학','깊이 우선 탐색', '']

In [37]:
problem_algorithm = get_problem_algorithm_list()
for algorithm in wanted_algorithm_list:
    if(algorithm not in problem_algorithm):
        print("Invalid algorithm")

In [41]:
user_id = 'abc'
a_query = "INSERT INTO update_user_table (user_id) VALUES (%s) ON DUPLICATE KEY UPDATE user_id = user_id"
cursor.execute(a_query, (user_id,))
conn.commit()

In [42]:
update_user_id_list_query = 'SELECT user_id FROM update_user_table'
cursor.execute(update_user_id_list_query)
update_user_id_list = cursor.fetchall()

In [44]:
update_user_id_list = [u[0] for u in update_user_id_list]

In [45]:
update_user_id_list

['abc', 'llsy159']

In [55]:
delete_user_id_query = "DELETE FROM update_user_table WHERE user_id = %s;"
cursor.execute(delete_user_id_query, (user_id,))
conn.commit()

In [57]:
import ast

In [61]:
ast.literal_eval('['']')

[]